In [1]:
from stardist.plot import render_label
from csbdeep.utils import normalize
from stardist.models import StarDist2D
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile as tiff
import skimage.io as io
import skimage.segmentation

In [8]:
def StarDist_segmentor(in_dir, save_dir):
    '''
    Produces binary masks for evaluation using StarDist segementation algorithm

    Parameters
    in_dir: str
        the path to the directory containing the grayscaled DAPI channel images of various fields
    save_dir: str
        the path to the directory where the binary masks will be saved. Each tiff image in the input directory will
        have a corresponding binary mask in the save directory
    '''
    
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the grayscaled DAPI channel of each field as an array
            DAPI_gray = io.imread(os.path.join(in_dir, file), as_gray=True)
            # loads pretrained stardist model
            model = StarDist2D.from_pretrained('2D_versatile_fluo')
            # predicting nuclei using the StarDist model
            normalized_img = normalize(DAPI_gray)
            label_mask, _ = model.predict_instances(normalized_img)
            # Post-processing of prediction label arrays into binary masks for evaluation. The overall effect of
            # step is that nuclei that are sparse are unchanged but nuclei that are touching are eroded by one
            # pixel where they touch other nuclei. This is necessary for binary connected component analysis during
            # F1-score calculation for segmentation evaluation.
            # Finding the pixels that are touching between any two nuclei
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

In [10]:
# Replace the strings for in_dir and save_dir to paths on the local machine
in_dir = r'C:\Users\mrl\Desktop\nuclear_seg\DAPI_grayscale'
save_dir = r'C:\Users\mrl\Desktop\nuclear_seg\StarDist_Binary_mask'
# To create binary masks using StarDist segmentation algorithm
StarDist_segmentor(in_dir, save_dir)

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=